In [ ]:
#!pip install wwf timm -qqq

In [ ]:
#downgrade pytorch because 1.9.1 is causing conflicts
!pip install --user /kaggle/input/fastaimaster/torch-1.9.0-cp37-cp37m-manylinux1_x86_64.whl

In [ ]:
!pip install ../input/timm-package/timm-0.4.12-py3-none-any.whl

In [ ]:
!pip install ../input/wwf0013wheel/wwf-0.0.13-py3-none-any.whl

In [ ]:
from wwf.utils import *
import timm

In [ ]:
#state_versions(['fastai', 'fastcore'])

In [ ]:
import numpy as np
import os
import pandas as pd
from fastai.vision.all import *

In [ ]:
from timm import create_model
from fastai.vision.learner import _update_first_layer

In [ ]:

def create_timm_body(arch:str, pretrained=True, cut=None, n_in=3):
    "Creates a body from any model in the `timm` library."
    model = create_model(arch, pretrained=pretrained, num_classes=0, global_pool='')
    _update_first_layer(model, n_in, pretrained)
    if cut is None:
        ll = list(enumerate(model.children()))
        cut = next(i for i,o in reversed(ll) if has_pool_type(o))
    if isinstance(cut, int): return nn.Sequential(*list(model.children())[:cut])
    elif callable(cut): return cut(model)
    else: raise NamedError("cut must be either integer or function")

In [ ]:
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
        os.makedirs('/root/.cache/torch/hub/checkpoints/')

!cp '../input/effnetv2rwm/efficientnetv2_rw_m_agc-3d90cb1e.pth' '/root/.cache/torch/hub/checkpoints/efficientnetv2_rw_m_agc-3d90cb1e.pth'

In [ ]:
#body = create_timm_body('resnet18', pretrained=True)
body = create_timm_body('efficientnetv2_rw_m', pretrained=True)
#resnetv2_50x1_bitm_in21k
#efficientnetv2_rw_t

In [ ]:
path = Path('../input/petfinder-pawpularity-score')

In [ ]:
train = pd.read_csv(path/'train.csv')

In [ ]:
train.head()

Added the next two rows for new getx

In [ ]:
train['image'] = train['Id'].apply(lambda x: 'train/{}.jpg'.format(x))

In [ ]:
train = train.drop(columns=['Id'])
train = train.sample(frac=1).reset_index(drop=True)
train.head(10)

In [ ]:
blocks = [ImageBlock, RegressionBlock()]

In [ ]:
item_tfms = [Resize(320)];# size should be bigger

In [ ]:
batch_tfms = [*aug_transforms(flip_vert=True,size=(224)), Normalize.from_stats(*imagenet_stats)]

In [ ]:
bs=8

In [ ]:
splitter = RandomSplitter()

In [ ]:
def get_x(row): return path/row['image']

In [ ]:
pets = DataBlock(blocks =blocks,
                  get_x = get_x,
                  get_y = ColReader('Pawpularity'),
                  splitter =splitter,
                  item_tfms=item_tfms,
                  batch_tfms = batch_tfms,
                  )

In [ ]:
dls = pets.dataloaders(train, bs=bs)

In [ ]:
dls.show_batch(nrows=2,ncols=1,figsize=(10,10))

In [ ]:
nf = num_features_model(body); nf

In [ ]:
head = create_head(nf, dls.c, concat_pool=True)

In [ ]:
net = nn.Sequential(body, head)

In [ ]:
learn = Learner(dls, net, splitter=default_split)

In [ ]:
learn.freeze()
frozen = filter(lambda p: not p.requires_grad, learn.model.parameters())
frozen = sum([np.prod(p.size()) for p in frozen])
model_parameters = filter(lambda p: p.requires_grad, learn.model.parameters())
unfrozen = sum([np.prod(p.size()) for p in model_parameters])

In [ ]:
frozen, unfrozen

In [ ]:
def create_timm_model(arch:str, n_out, cut=None, pretrained=True, n_in=3, init=nn.init.kaiming_normal_, custom_head=None,
                     concat_pool=True, **kwargs):
    "Create custom architecture using `arch`, `n_in` and `n_out` from the `timm` library"
    body = create_timm_body(arch, pretrained, None, n_in)
    if custom_head is None:
        nf = num_features_model(nn.Sequential(*body.children()))
        head = create_head(nf, n_out, concat_pool=concat_pool, **kwargs)
    else: head = custom_head
    model = nn.Sequential(body, head)
    if init is not None: apply_init(model[1], init)
    return model

# Cell
from fastai.vision.learner import _add_norm

#exports
def timm_learner(dls, arch:str, loss_func=None, pretrained=True, cut=None, splitter=None,
                y_range=None, config=None, n_out=None, normalize=True, **kwargs):
    "Build a convnet style learner from `dls` and `arch` using the `timm` library"
    if config is None: config = {}
    if n_out is None: n_out = get_c(dls)
    assert n_out, "`n_out` is not defined, and could not be inferred from data, set `dls.c` or pass `n_out`"
    if y_range is None and 'y_range' in config: y_range = config.pop('y_range')
    model = create_timm_model(arch, n_out, default_split, pretrained, y_range=y_range, **config)
    learn = Learner(dls, model, loss_func=loss_func, splitter=default_split, **kwargs)
    if pretrained: learn.freeze()
    return learn

In [ ]:
learn = timm_learner(dls, 'efficientnetv2_rw_m', metrics=[rmse])

In [ ]:
frozen = filter(lambda p: not p.requires_grad, learn.model.parameters())
frozen = sum([np.prod(p.size()) for p in frozen])
model_parameters = filter(lambda p: p.requires_grad, learn.model.parameters())
unfrozen = sum([np.prod(p.size()) for p in model_parameters])

In [ ]:
frozen, unfrozen

In [ ]:
learn.fine_tune(4)

In [ ]:
learn.save('stage_1')

In [ ]:
learn.load('stage_1')

In [ ]:
learn.unfreeze()

In [ ]:
lrs = learn.lr_find(suggest_funcs=(minimum, steep, valley, slide))

In [ ]:
lrs.slide

In [ ]:
lrs.valley

In [ ]:

learn.fine_tune(4, base_lr=lrs.valley) 

In [ ]:
learn.save('stage_2')

In [ ]:
learn.load('stage_2');

Inference

In [ ]:
sample_df = pd.read_csv(path/'sample_submission.csv')
sample_df.head()

In [ ]:
test_df = pd.read_csv(path/'test.csv')
test_df.head()

In [ ]:
test_imgs = '../input/petfinder-pawpularity-score/test'

In [ ]:
test_df['image'] = test_df['Id'].apply(lambda x: 'test/{}.jpg'.format(x))

In [ ]:
test_df.head()

In [ ]:
sample_copy = test_df.copy()

In [ ]:
sample_copy.head()

In [ ]:
test_dl = learn.dls.test_dl(sample_copy)

In [ ]:
test_dl.show_batch()

In [ ]:
preds = learn.get_preds(dl=test_dl)

In [ ]:
import os 
test_data = pd.DataFrame()

In [ ]:
test_data['Id'] = [img[:-4]for img in os.listdir(test_imgs)]
test_data

In [ ]:
# Now adding the pawpularity score to our dataframe 
test_data['Pawpularity'] = preds[0].detach().numpy()
test_data

In [ ]:
# Making the submission
test_data.to_csv('submission.csv' , index=False)